In [1]:
import pandas as pd
dataset=pd.read_csv('zeolite_Amorphous.csv', index_col=0, header=0)
x=pd.concat([dataset.iloc[:, 0:5], dataset.iloc[:, 6:8]], axis=1)
y=dataset.iloc[:, 8]
from sklearn.model_selection import train_test_split
import numpy as np
rs=np.arange(1, 101, 1)
splits=[]

for i in rs:
    x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.4, shuffle=True, stratify=y, random_state=i)
    splits.append((i, x_train, x_test, y_train, y_test))

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
ratio_of_xs = np.arange(0.1, 1.1, 0.1)
results = []

for i, x_train, x_test, y_train, y_test in splits:
    accuracy_oob=[]
    all_ratio_of_xs=[]
    for ratio_of_x in ratio_of_xs:
        model_oob=RandomForestClassifier(n_estimators=1000, max_features=ratio_of_x, oob_score=True, random_state=5)
        model_oob.fit(x_train, y_train)
        accuracy_oob.append(model_oob.oob_score_)
        all_ratio_of_xs.append(ratio_of_x)
    optimal_x=all_ratio_of_xs[accuracy_oob.index(max(accuracy_oob))]

    model=RandomForestClassifier(n_estimators=1000, max_features=optimal_x, oob_score=True, random_state=5)
    model.fit(x_train, y_train)
    estimated_y_test=pd.DataFrame(model.predict(x_test))
    accuracy_test=metrics.accuracy_score(y_test, estimated_y_test)
    results.append({"random_state":i, "accuracy":accuracy_test})
    df_results=pd.DataFrame(results)
    df_results

In [2]:
df_results

,random_state,accuracy
0,1,0.744681
1,2,0.792553
2,3,0.765957
3,4,0.755319
4,5,0.792553
...,...,...
95,96,0.765957
96,97,0.813830
97,98,0.776596
98,99,0.792553


In [3]:
df_results.to_csv('accuracy_rf_0.4.csv')